In [1]:
pip install autokeras -q

     |████████████████████████████████| 174kB 7.8MB/s 
     |████████████████████████████████| 71kB 5.0MB/s 


## Customize a block for tuning the number of units

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

def build_mlp():
    input_node = keras.Input(shape=(20,))
    
    output_node = layers.Dense(units=32, activation='relu')(input_node)
    output_node = layers.Dense(units=32, activation='relu')(output_node)
    output_node = layers.Dense(units=1, activation='sigmoid')(output_node)
    
    model = keras.Model(input_node, output_node)
    return model 

mlp_model = build_mlp()

In [ ]:
import autokeras as ak
import tensorflow as tf
from tensorflow.keras import layers


class MlpBlock(ak.Block):

  def build(self, hp, inputs):
  
    input_node = tf.nest.flatten(inputs)[0]
    
    units_1 = hp.Int(name='units_1', min_value=32, max_value=512, step=32)
    
    units_2 = hp.Int(name='units_2', min_value=32, max_value=512, step=32)  
        
    output_node = layers.Dense(units=units_1, activation='relu')(input_node)
    
    output_node = layers.Dense(units=units_2, activation='relu')(output_node)
    
    return output_node

## Customize a block for tuning different types of hyperparameters

In [ ]:
import autokeras as ak
import tensorflow as tf
from tensorflow.keras import layers


class MlpBlock(ak.Block):
  def build(self, hp, inputs):
    output_node = tf.nest.flatten(inputs)[0]
    for i in range(hp.Choice("num_layers", [1, 2, 3])):
      output_node = layers.Dense(units=hp.Int('units_' + str(i),
                                              min_value=32,
                                              max_value=512,
                                              step=32),
                                 activation='relu')(output_node)
    return output_node

In [ ]:
import autokeras as ak
import tensorflow as tf
from tensorflow.keras import layers


class MlpBlock(ak.Block):
  def build(self, hp, inputs):
    output_node = tf.nest.flatten(inputs)[0]
    for i in range(hp.Choice("num_layers", [1, 2, 3])):
      output_node = layers.Dense(units=hp.Int('units_' + str(i),
                                              min_value=32,
                                              max_value=512,
                                              step=32),
                                 activation='relu')(output_node)
    if hp.Boolean("dropout"):
      output_node = layers.Dropout(rate=hp.Float("dropout_rate",
                                                 min_value=0,
                                                 max_value=1))
    return output_node

## Using the customized block to create an AutoML pipeline

In [ ]:
import kerastuner as kt
hp = kt.HyperParameters()
inputs = tf.keras.Input(shape=(20,))
MlpBlock().build(hp, inputs)

<KerasTensor: shape=(None, 32) dtype=float32 (created by layer 'dense_3')>

In [ ]:
import numpy as np
x_train = np.random.rand(100, 20)
y_train = np.random.rand(100, 1)
x_test = np.random.rand(100, 20)

input_node = ak.StructuredDataInput()
output_node = MlpBlock()(input_node)
output_node = ak.RegressionHead()(output_node)
auto_model = ak.AutoModel(input_node, output_node, max_trials=1, overwrite=True)
auto_model.fit(x_train, y_train, epochs=1)

Trial 1 Complete [00h 00m 01s]
val_loss: 0.12918557226657867

Best val_loss So Far: 0.12918557226657867
Total elapsed time: 00h 00m 01s
INFO:tensorflow:Oracle triggered exit
4/4 [==============================] - 0s 3ms/step - loss: 0.1318 - mean_squared_error: 0.1318
INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


In [ ]:
auto_model.predict(x_test).shape

(100, 1)

In [ ]:
auto_model.tuner.search_space_summary()

Search space summary
Default search space size: 6
mlp_block_2/num_layers (Choice)
{'default': 1, 'conditions': [], 'values': [1, 2, 3], 'ordered': True}
mlp_block_2/units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
mlp_block_2/dropout (Boolean)
{'default': False, 'conditions': []}
regression_head_1/dropout (Choice)
{'default': 0, 'conditions': [], 'values': [0.0, 0.25, 0.5], 'ordered': True}
optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'sgd', 'adam_weight_decay'], 'ordered': False}
learning_rate (Choice)
{'default': 0.001, 'conditions': [], 'values': [0.1, 0.01, 0.001, 0.0001, 2e-05, 1e-05], 'ordered': True}
